In [9]:
# Dependencies
import requests
from config_env import api_key
import time
import json
import os
from IPython.core.display import display, HTML
import pymongo
from pymongo import MongoClient
#for scraping article
from bs4 import BeautifulSoup as bs


In [10]:
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Store a search term
query = "Venezuela"
param = 'fq=news_desk:(Foreign)&fq=source:(The New York Times)'
# Search for articles published between a begin and end date
begin_date = "20140101"
end_date = "20190101"
page = '&page='

query_url = f"{url}{param}&api-key={api_key}&q={query}&\
facet_field=snippet&begin_date={begin_date}&end_date={end_date}{page}"

In [22]:
output_list = []
# Retrieve articles

for n in range(1, 2):
    url=query_url + str(n)
    try:
        articles = requests.get(url).json()
        print(f"Getting data from {url}")
        articles_list = [article for article in articles["response"]["docs"]]
    
        for item in articles_list:
            list_dict = {}
            list_dict["snippet"] = item["snippet"]
            list_dict["lede"] = item["lead_paragraph"]
            list_dict["url"] = item["web_url"]
            list_dict["headline"] = item["headline"]["main"]
            list_dict["keywords"] = item["keywords"]
            list_dict["pub_date"] = item["pub_date"]
            output_list.append(list_dict)
        
        time.sleep(6)
    except KeyError:
        print(f"Error with {n}")
        pass



Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=LptynSqmgrYWGBusejIrH6N4Ci8tHqD5&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=1


In [23]:
len(output_list)

10

In [24]:
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client.venezuela   

Mongo version 3.7.2


In [25]:
db.nyt_news.insert_many(output_list)

In [26]:
news = db.nyt_news.find()
news

In [27]:
# Get a list of URLs to scrape
urls = []
for item in news:
    urls.append(item['url'])

In [28]:
len(urls)

50

In [29]:
out_list = []
# Visit the NYT site and get the body copy
for item in urls:
    output_dict = {}
    response = requests.get(item, verify=False)
    soup = bs(response.text, 'lxml')
    body_copy = soup.find_all('div', class_='css-1ygdjhk evys1bk0')
    output_dict['body_copy'] = str(body_copy)
    db.nyt_news.update_one({'url':item}, {'$set': output_dict})
    time.sleep(2)


/Users/lauraweislo/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lauraweislo/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lauraweislo/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lauraweislo/anaconda3/lib/python3.7/site-packages/urllib3/conne

/Users/lauraweislo/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lauraweislo/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lauraweislo/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lauraweislo/anaconda3/lib/python3.7/site-packages/urllib3/conne

In [19]:
output_dict


{'body_copy': 'None'}